In [ ]:
# pip install awscli

In [5]:
####
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time
from time import sleep

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys


####SCRAPE THE WEBSITE######

# object of Options class
op = Options()
op.headless = True

######## New way
driver = webdriver.Chrome("D:\JITHU\Yeshiva - USA\Fall - 2023\Information Architectures\Assignments\M10_WebScrapper\Scripts\chromedriver-win64\chromedriver.exe", options=op)
driver.maximize_window()
sleep(2)


driver.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')
sleep(3)


TextBox1 = '//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[3]/td[2]/input'
start = driver.find_element_by_xpath(TextBox1)
start.clear()
sleep(1)
start.send_keys('0')
sleep(1)

submit_button = '//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]'
submit = driver.find_element_by_xpath(submit_button)
submit.click()

sleep(3)



#identify the table to scrape
table_xpath = '/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]'
table_element = driver.find_element_by_xpath(table_xpath)

# Find rows in the table
rows = table_element.find_elements_by_tag_name("tr")
columns = table_element.find_elements_by_tag_name("th")

table_data = []
for row in rows:
    # Find cells in the row
    cells = row.find_elements_by_tag_name("td")
    row_data = [cell.text for cell in cells]
    table_data.append(row_data)
    
# Now table_data contains the text from the table cells
# Create a DataFrame from the extracted data
df = pd.DataFrame(table_data, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
# If the first row of your data is headers, you can do the following:
# df.columns = df.iloc[0]  # Set the first row as column header
df = df[1:]  # Drop the first row since it's now the header
df

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,038249,15-68-83,NO DATA AVAILABLE,NFP,CHARLOTTESVILLE,VA
3,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
4,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
5,"10,000 Entrepreneurs, Inc.",47-42-99,832744892,NFP,MANCHESTER,NH
6,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
7,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
8,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY
9,1000 Islands Regional Local Development Corpor...,47-98-37,841988435,NFP,"WATERTOWN, NY",
10,"1000 Steps Fellows, Inc.",47-01-23,824598461,NFP,"BROOKLYN, NY",


# Checking Existing buckets

In [6]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')
    

Existing buckets:
  assignment-customerreviews
  information-arch-jetendra-assignment-8a


# Create an s3 bucket for that file

In [7]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [8]:
create_bucket('information-arch-jetendra-assignment-m10-webscrapper')

True

In [9]:
############# Confirming bucket is created

# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')
    

Existing buckets:
  assignment-customerreviews
  information-arch-jetendra-assignment-8a
  information-arch-jetendra-assignment-m10-webscrapper


'charities_bureau_scrape_Group3_20231213085149.csv'

In [14]:
from botocore.exceptions import NoCredentialsError

# Assuming you have a DataFrame called df
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
file_name = 'charities_bureau_scrape_Group3_{0}.csv'.format(datetime)


# Convert DataFrame to CSV and save it locally
df.to_csv(file_name, index=False)

# Upload the CSV to S3
bucket_name = 'information-arch-jetendra-assignment-m10-webscrapper'

s3 = boto3.client('s3')

try:
    s3.upload_file(file_name, bucket_name, file_name)
    print(f'{file_name} uploaded to {bucket_name}.')
except FileNotFoundError:
    print(f'The file {file_name} was not found.')
except NoCredentialsError:
    print('Credentials not available.')

charities_bureau_scrape_Group3_20231213085219.csv uploaded to information-arch-jetendra-assignment-m10-webscrapper.


In [ ]:
# ###LOAD THE FILE INTO S3####
# # prepare csv file name   
# pathname = 's3:/NAME_OF_YOUR_S3_BUCKET/'#specify location of s3:/{my-bucket}/
# filename= 'charities_bureau_scrape_' #name of your group
# datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
# filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

# #load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
# df.to_csv(filenames3, header=True, line_terminator='\n') 

# #print success message
# print("Successfull uploaded file to location:"+str(filenames3))